In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random

import warnings
warnings.filterwarnings('ignore')

drive_path = '/content/drive/MyDrive/Kuliah/Tugas Akhir/Final Project Shared Folder'
data_path = "Dataset/Data Versioning/"
model_path = "Model/ML Model/"
data_version = "Trained_V1.csv"
base_url = "https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?CycleBeginYear=2017"
dataset_names = ['Demographics', 'Dietary', 'Examination', 'Laboratory', 'Questionnaire']

Mounted at /content/drive


In [ ]:
df_raw = pd.read_csv(os.path.join(drive_path, data_path+data_version), index_col = 0)
try:
  df_raw = df_raw.set_index('SEQN', drop=True)
  df_raw = df_raw.drop(columns = "Unnamed: 0")
except:
  pass
df_raw.head()

,Demog1_RIAGENDR,Demog1_RIDAGEYR,Demog1_RIDRETH3,Demog1_DMDMARTL,Demog1_DMDHHSIZ,Demog1_DMDFMSIZ,Demog1_DMDHHSZA,Demog1_DMDHHSZB,Demog1_DMDHHSZE,Demog1_DMDEDUC,...,Dieta2_DR1TCALC,Exami1_SysPulse,Exami1_DiaPulse,MCQ010,MCQ160B,MCQ220,MCQ300C,MCQ300A,MCQ366A,MCQ366B
SEQN,,,,,,,,,,,,,,,,,,,,,
93703.0,2.0,2.0,6.0,5.0,5.0,5.0,3.000000e+00,5.397605e-79,5.397605e-79,1.0,...,893.39645,86.000000,40.000000,2.0,1.0,9.0,9.0,9.0,9.0,9.0
93704.0,1.0,2.0,3.0,5.0,4.0,4.0,2.000000e+00,5.397605e-79,5.397605e-79,1.0,...,700.00000,88.000000,42.000000,2.0,1.0,9.0,9.0,9.0,9.0,9.0
93705.0,2.0,66.0,4.0,3.0,1.0,1.0,5.397605e-79,5.397605e-79,1.000000e+00,2.0,...,314.00000,69.961686,69.961686,1.0,2.0,2.0,1.0,2.0,2.0,2.0
93706.0,1.0,18.0,6.0,5.0,5.0,5.0,5.397605e-79,5.397605e-79,1.000000e+00,4.0,...,869.00000,73.333333,73.333333,2.0,1.0,9.0,9.0,9.0,2.0,2.0
93707.0,1.0,13.0,7.0,5.0,7.0,7.0,5.397605e-79,3.000000e+00,5.397605e-79,1.0,...,535.00000,47.333333,47.333333,2.0,1.0,9.0,9.0,9.0,9.0,9.0


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def prep_data(df):
  X = df.copy().drop(columns=['MCQ010', 'MCQ160B', 'MCQ220', 'MCQ300C', 'MCQ300A', 'MCQ366A', 'MCQ366B'])
  y = df['MCQ160B']

  y = y.replace({2: 0})

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_val = scaler.transform(X_val)

  X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
  X_val = np.array(X_val).reshape(X_val.shape[0], X_val.shape[1], 1)

  print("Train: ", X_train.shape, " ", y_train.shape)
  print("Val: ", X_val.shape, " ", y_val.shape)
  print("Column Used: ", X.columns.tolist())

  return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = prep_data(df_raw)

Train:  (6477, 78, 1)   (6477,)
Val:  (2777, 78, 1)   (2777,)
Column Used:  ['Demog1_RIAGENDR', 'Demog1_RIDAGEYR', 'Demog1_RIDRETH3', 'Demog1_DMDMARTL', 'Demog1_DMDHHSIZ', 'Demog1_DMDFMSIZ', 'Demog1_DMDHHSZA', 'Demog1_DMDHHSZB', 'Demog1_DMDHHSZE', 'Demog1_DMDEDUC', 'Demog1_INDIN2', 'Labor1_LBXTC', 'Exami1_BPXPLS', 'Exami1_BPXPULS', 'Exami2_BMXWT', 'Exami2_BMXHT', 'Exami2_BMXBMI', 'Quest1_ALQ111', 'Quest2_BPQ020', 'Quest3_CDQ008', 'Quest3_CDQ010', 'Quest4_CBD111', 'Quest4_CBD121', 'Quest7_DIQ010', 'Quest9_DLQ050', 'Quest10_ECQ020', 'Quest11_HIQ011', 'Quest12_HEQ010', 'Quest12_HEQ030', 'Quest14_INQ020', 'Quest14_INQ012', 'Quest14_IND235', 'Quest15_KIQ022', 'Quest15_KIQ026', 'Quest17_DPQ020', 'Quest17_DPQ030', 'Quest17_DPQ040', 'Quest18_OCQ210', 'Quest19_PAQ635', 'Quest20_PFQ020', 'Quest20_PFQ061B', 'Quest20_PFQ061C', 'Quest20_PFQ061H', 'Quest22_SMQ020', 'Quest22_SMQ890', 'Quest22_SMQ900', 'Quest3_CDQ009', 'Dieta1_DRDINT', 'Dieta1_DR1DAY', 'Dieta1_DR1FS', 'Dieta1_DR1IFDCD', 'Dieta1_DR1IGR

In [ ]:
y_val.value_counts()

0.0    1609
1.0    1168
Name: MCQ160B, dtype: int64

# Create Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

class CNNModel(BaseEstimator, ClassifierMixin):
  def __init__(
      self,
      input_shape,
      layer_metadata,
      convolution_part=['conv1d'],
      intermediate_part=['flatten'],
      dense_part=['dense'],
      convolution_hyperparameter=[{'filters':16, 'kernel_size':3, 'activation':'relu'}],
      intermediate_hyperparameter=[{}],
      dense_hyperparameter=[{'units':32, 'activation':'relu'}],
      optimizer='adam'
  ):
    self.input_shape = input_shape
    self.layer_metadata = layer_metadata
    self.convolution_part = convolution_part
    self.intermediate_part = intermediate_part
    self.dense_part = dense_part
    self.convolution_hyperparameter = convolution_hyperparameter
    self.intermediate_hyperparameter = intermediate_hyperparameter
    self.dense_hyperparameter = dense_hyperparameter
    self.optimizer = optimizer
    self.model = self.build_model(input_shape, convolution_part, intermediate_part, dense_part, convolution_hyperparameter, intermediate_hyperparameter, dense_hyperparameter, optimizer)



  def build_model(
      self,
      input_shape,
      convolution_part,
      intermediate_part,
      dense_part,
      convolution_hyperparameter,
      intermediate_hyperparameter,
      dense_hyperparameter,
      optimizer,
  ):

    def fix_hyperparameter_type(layer, hyperparameter, layer_metadata, warnings=True):
      for layer_info in layer_metadata:
        if(layer == layer_info['layer']):
          for params in layer_info['hyperparameter']:
            val = hyperparameter.get(params['param'], params['default'])
            if(type(val) != params['type']):
              if(warnings): print("Hyperparameter ", params['param'], " Have Invalid Data Type! Using Default..")
              hyperparameter[params['param']] = params['default']
            else:
              hyperparameter[params['param']] = val
        else:
          pass
      return hyperparameter

    model = models.Sequential()
    model.add(layers.Input(input_shape))

    # Convolution Part
    for layer, hyperparameter in zip(convolution_part, convolution_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'conv1d'):
        model.add(layers.Conv1D(filters=hyperparameter['filters'], kernel_size=hyperparameter['kernel_size'], activation=hyperparameter['activation']))
      elif(layer == 'maxpooling1d'):
        model.add(layers.MaxPooling1D(pool_size=hyperparameter['pool_size']))
      else:
        print("'Convolution Part' Layer Invalid!")
        return None

    # Intermediate Part
    for layer, hyperparameter in zip(intermediate_part, intermediate_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'flatten'):
        model.add(layers.Flatten())
      else:
        print("'Intermediate Part' Layer Invalid!")
        return None

    # Dense Part
    for layer, hyperparameter in zip(dense_part, dense_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'dense'):
        model.add(layers.Dense(units=hyperparameter['units'], activation=hyperparameter['activation']))
      else:
        print("'Dense Part' Layer Invalid!")
        return None

    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

    return model

  def fit(self, X_train, y_train, X_test, y_test, epochs, verbose=1):
    self.model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=verbose)

  def score(self, X, y):
    loss, accuracy = self.model.evaluate(X, y, verbose=0)
    loss_inverse = 1/(loss+1e-20)

    return loss_inverse

  def evaluate(self, X, y):
    # Evaluate the model on the test set
    loss, accuracy = self.model.evaluate(X, y, verbose=0)

    print(f"Loss: %.3f%%" % (loss*100) )
    print(f"Accuracy: %.3f%%" % (accuracy*100) )

    return loss, accuracy

  def summary(self):
    self.model.summary()

  def predict(self, X):
    return self.model.predict(X, verbose=0)

  def save(self, path, save_format='keras'):
    return self.model.save(path, save_format)

In [ ]:
layer_metadata = [
  {'layer': 'conv1d', 'hyperparameter': [
      {'param':'filters', 'type':int, 'default':16},
      {'param':'kernel_size', 'type':int, 'default':3},
      {'param':'activation', 'type':str, 'default':'linear'}
    ]
  },
  {'layer': 'maxpooling1d', 'hyperparameter': [
      {'param':'pool_size', 'type':int, 'default':2}
    ]
  },
  {'layer': 'flatten', 'hyperparameter': [

    ]
  },
  {'layer': 'dense', 'hyperparameter': [
      {'param':'units', 'type':int, 'default':8},
      {'param':'activation', 'type':str, 'default':'linear'}
    ]
  }
]

In [ ]:
tf.random.set_seed(37)
random.seed(37)

model_schema = {}

# must include
model_schema['input_shape'] = X_train.shape[1:]
model_schema['layer_metadata'] = layer_metadata

# option
model_schema['convolution_part'] = ['conv1d', 'maxpooling1d', 'conv1d', 'maxpooling1d']
model_schema['convolution_hyperparameter'] =  [{'filters':32, 'kernel_size':3}, {}, {'filters':16, 'kernel_size':3, 'activation':'relu'}, {}]

model_schema['dense_part'] = ['dense', 'dense', 'dense']
model_schema['dense_hyperparameter'] =  [ {'units':32, 'activation':'relu'}, {'units':16, 'activation':'relu'}, {'units':16, 'activation':'relu'}]

model = CNNModel(**model_schema)

model.summary()

model.fit(X_train, y_train, X_val, y_val, epochs=20)
model.evaluate(X_val, y_val)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 76, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1  (None, 38, 32)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 36, 16)            1552      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 18, 16)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 288)               0         
                                                                 
 dense (Dense)               (None, 32)                9

(0.06598912179470062, 0.9801944494247437)

In [ ]:
# model.save(os.path.join(drive_path, model_path+'model_cnn_v1.h5'), save_format='h5')

In [ ]:
# new_model = tf.keras.models.load_model(os.path.join(drive_path, model_path+'model_cnn_v1.h5'))
# new_model.summary()

# loss, accuracy = new_model.evaluate(X_val, y_val, verbose=0)

# print(f"Loss: %.3f%%" % (loss*100) )
# print(f"Accuracy: %.3f%%" % (accuracy*100) )

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 76, 32)            128       
                                                                 
 max_pooling1d_20 (MaxPooli  (None, 38, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 36, 16)            1552      
                                                                 
 max_pooling1d_21 (MaxPooli  (None, 18, 16)            0         
 ng1D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 288)               0         
                                                                 
 dense_40 (Dense)            (None, 32)              

In [ ]:
# model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 76, 32)            128       
                                                                 
 max_pooling1d_20 (MaxPooli  (None, 38, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 36, 16)            1552      
                                                                 
 max_pooling1d_21 (MaxPooli  (None, 18, 16)            0         
 ng1D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 288)               0         
                                                                 
 dense_40 (Dense)            (None, 32)              